In [4]:
import numpy as np           # Handle matrices
from vizdoom import *        # Doom Environment

import random                # Handling random number generation
import time                  # Handling time calculation

from collections import deque# Ordered collection with ends
import matplotlib.pyplot as plt # Display graphs

from torchvision import transforms, utils
import torch

In [5]:
def create_environment():
    game = DoomGame()
    
    # Load the correct configuration
    game.load_config("basic.cfg")
    
    game.init()
    
    # Here our possible actions
    left = [1, 0, 0]
    right = [0, 1, 0]
    shoot = [0, 0, 1]
    possible_actions = [left, right, shoot]
    
    return game, possible_actions

## Pre-processing
* doom returns numpy array for the image
* Crop-screen frame[30:-10,30:-30] (just removes the roof)
* normalize / 255
* resize to (84,84)

In [6]:
test_environment()

<class 'numpy.ndarray'>
[1, 0, 0]
	reward: -1.0
<class 'numpy.ndarray'>
[1, 0, 0]
	reward: -1.0
<class 'numpy.ndarray'>
[0, 0, 1]
	reward: -1.0
<class 'numpy.ndarray'>
[1, 0, 0]
	reward: -1.0
<class 'numpy.ndarray'>
[1, 0, 0]
	reward: -1.0
<class 'numpy.ndarray'>
[0, 1, 0]
	reward: -1.0
<class 'numpy.ndarray'>
[0, 0, 1]
	reward: -6.0
<class 'numpy.ndarray'>
[1, 0, 0]
	reward: -1.0
<class 'numpy.ndarray'>
[0, 1, 0]
	reward: -1.0
<class 'numpy.ndarray'>
[0, 0, 1]
	reward: -1.0
<class 'numpy.ndarray'>
[0, 0, 1]
	reward: -1.0
<class 'numpy.ndarray'>
[0, 1, 0]
	reward: -1.0
<class 'numpy.ndarray'>
[1, 0, 0]
	reward: -1.0
<class 'numpy.ndarray'>
[0, 1, 0]
	reward: -1.0
<class 'numpy.ndarray'>
[0, 0, 1]
	reward: -1.0
<class 'numpy.ndarray'>
[1, 0, 0]
	reward: -1.0
<class 'numpy.ndarray'>
[0, 1, 0]
	reward: -1.0
<class 'numpy.ndarray'>
[0, 1, 0]
	reward: -1.0
<class 'numpy.ndarray'>
[1, 0, 0]
	reward: -1.0
<class 'numpy.ndarray'>
[1, 0, 0]
	reward: -1.0
<class 'numpy.ndarray'>
[0, 0, 1]
	rewar

ViZDoomUnexpectedExitException: Controlled ViZDoom instance exited unexpectedly.